In [1]:
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv

#scientific computing
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats

Makes design files for the ser, loc, and sim experiments

In [76]:
#preliminary stuff
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'scripts/sub_cb_mappings.txt'
os.chdir(home_dir)

time_to_drop = 12.0 #number of seconds of scans to drop

#get subject list
sub_list = []
sub_to_cb = {}
with open(subj_file) as f:
    for line in f:
        (key,val) = line.strip().split(' ') 
        sub_list.append('fd_' + key)
        sub_to_cb['fd_' + key] = int(val)

In [77]:
##get timing file for object localizer
def load_timing_file(run,time_to_drop):
    loc_file = home_dir + '/loc_timing/script_kidLoc_2Hz_run' + str(run)
    trial_type = []
    time = []
    with open(loc_file) as f:
        for n,line in enumerate(f):
            l = line.strip('\n').split('\t')
            if len(l)==5: #ignore file header and footer
                trial_type.append(int(l[2]))
                time.append(float(l[1]) - time_to_drop)
    return trial_type, time

In [82]:
#trial type dictionary for object localizer
def trial_type_dict():
    return {0:'scramble', 1:'face-adult', 2:'face-child', 3:'body', 4:'limb', 5:'car',
            6:'guitar', 7:'place', 8:'house', 9:'word', 10:'number'}

In [83]:
def write_loc_file(design_dir,cb):
    trial_dict = trial_type_dict()
    duration = 4 #all blocks are 4s
    
    #set up run counterbalance
    if cb==1:
        run_dict = {1: '1', 2:'2'}
    else:
        run_dict = {1: '2', 2:'1'}
    
    #prepare out file
    out_file = design_dir + 'loc.csv'
    if os.path.exists(out_file):
        os.remove(out_file)
    out_f = open(out_file,'w')
    out_f.write('run,condition,onset,duration\n')

    for run in range(1,3):
        trial_type, time = load_timing_file(run,time_to_drop)

        #write out file
        for n,trial in enumerate(trial_type):
            if n > 0 and trial != trial_type[n-1]: #start of a new trial
                if trial_dict[trial] != 'scramble': #not a condition
                    new_line = ','.join([run_dict[run],trial_dict[trial],str(time[n]),str(4)+'\n'])
                    out_f.write(new_line)
    out_f.close()
    return

In [84]:
#make design folder
for sub in sub_list:
    design_dir = home_dir + 'data/' + sub + '/design/'
    if not os.path.exists(design_dir):
        os.makedirs(design_dir)
    
    #write file
    write_loc_file(design_dir, sub_to_cb[sub])
    